In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, roc_auc_score, confusion_matrix

# ---------------------------------------------------------
# 1. 데이터 로드 (타겟이 이미 구해진 데이터셋)
# ---------------------------------------------------------

# Target_data_산출.ipynb에서 생성된 최종 데이터를 로드합니다.
# 경로와 파일명은 상림이가 저장한 설정에 맞게 수정하면 됩니다.
df_final = pd.read_csv('Target_data_산출.csv', low_memory=False)

# VIP 세그먼트 분리 (Final_Total_Score 기준 하위 20%)
threshold_vip = df_final['Final_Total_Score'].quantile(0.2)
df_vip = df_final[df_final['Final_Total_Score'] <= threshold_vip].copy()

# ---------------------------------------------------------
# 2. 변수 중요도 산출 (변수중요도확인 로직)
# ---------------------------------------------------------

# 분석 제외 컬럼 설정
exclude_cols = ['발급회원번호', 'Target', 'Final_Total_Score', 'Score_Status_Total', 'Score_Slope_Total',
    'Score_BadDebt', 'Score_Delinq', 'Score_Activity', 'Score_Asset', 
    'Slope_Spend', 'Slope_Balance', 'Slope_Count', 'Risk_Count']
X_vip = df_vip.select_dtypes(include=[np.number]).drop(columns=[c for c in exclude_cols if c in df_vip.columns], errors='ignore')
y_vip = df_vip['Target']

# 데이터 정제 (무한대/결측치 처리 및 분산 0인 컬럼 제거)
X_vip = X_vip.replace([np.inf, -np.inf], 0).fillna(0)
selector = VarianceThreshold(threshold=0)
X_reduced = X_vip.loc[:, selector.fit(X_vip).get_support()]

# 중요도 산출용 LightGBM 학습
lgbm_imp = lgb.LGBMClassifier(n_estimators=200, random_state=42, verbose=-1)
lgbm_imp.fit(X_reduced, y_vip)

# 피처 중요도 결과 구성
importance_df = pd.DataFrame({'Feature': X_reduced.columns, 'Importance': lgbm_imp.feature_importances_})

# ---------------------------------------------------------
# 3. 상위 100개 컬럼 추출 (컬럼추출하기 로직)
# ---------------------------------------------------------

# 중요도 상위 100개 컬럼 리스트 추출
top_100_features = importance_df.sort_values(by='Importance', ascending=False).head(100)['Feature'].tolist()

# ---------------------------------------------------------
# 4. 최종 모델 학습 및 평가 (모델돌리기 로직)
# ---------------------------------------------------------

# 상위 100개 피처 데이터셋 구성
X_final = X_reduced[top_100_features]
y_final = y_vip

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=42, stratify=y_final)

# 최종 예측 모델 학습 (불균형 데이터 가중치 적용)
model_vip = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    is_unbalance=True,
    random_state=42,
    verbose=-1
)
model_vip.fit(X_train, y_train)

# 성능 평가 결과 출력
y_pred = model_vip.predict(X_test)
y_prob = model_vip.predict_proba(X_test)[:, 1]

print("[VIP 그룹 모델 성능 결과]")
print(classification_report(y_test, y_pred))
print(f"ROC-AUC: {roc_auc_score(y_test, y_prob):.4f}")

[VIP 그룹 모델 성능 결과]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

ROC-AUC: nan


/usr/local/Caskroom/miniconda/base/envs/myenv/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
